In [1]:
library(tidyverse)
# install.packages("rcrossref")
library(rcrossref)

-- Attaching packages --------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.4.1     v purrr   1.0.1
v tibble  3.1.8     v dplyr   1.1.0
v tidyr   1.3.0     v stringr 1.5.0
v readr   2.1.4     v forcats 1.0.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
# Make sure all columns of a tibble are being shown
options(repr.matrix.max.rows = Inf, repr.matrix.max.cols = Inf)

In [3]:
grants <- read_delim("2_GrantNumbersWithPublicatons.csv", delim = ";", col_names=TRUE)

Rows: 4353 Columns: 1
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl (1): GrantNumber

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
dim(grants)
head(grants)

[1] 4353    1

GrantNumber
<dbl>
149634
146722
150754
157023
172242
156019


In [5]:
publications <- read_delim("../raw/OutputdataScientificPublication.csv", 
                           col_select = c(
                               "GrantNumber",
                               "ScientificPublication_DOI",
                               "ScientificPublication_Year"
                           ), 
                           delim = ";", col_names=TRUE)

Rows: 173212 Columns: 3
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr (1): ScientificPublication_DOI
dbl (2): GrantNumber, ScientificPublication_Year

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
dim(publications)

[1] 173212      3

In [7]:
# Keep only publications with a DOI
publications <- filter(publications, !is.na(ScientificPublication_DOI))

In [8]:
dim(publications)

[1] 110781      3

In [9]:
# Keep only the publications of the grants we're interested in (i.e. the biomed grants!)
publications <- filter(publications, GrantNumber %in% grants$GrantNumber)

In [10]:
dim(publications)

[1] 31625     3

In [11]:
# Trim the DOIS
publications$ScientificPublication_DOI <- str_trim(publications$ScientificPublication_DOI)

In [12]:
# Apparently, some DOIs in the original data are not well-formed!!!

# substring <- "http"
# result <- publications[grepl(substring, publications$ScientificPublication_DOI), ]
# result

# substring <- "works"
# result <- publications[grepl(substring, publications$ScientificPublication_DOI), ]
# result

In [13]:
# We remove the most obvious mistakes

# http://dx.doi.org/
# https://doi.org/
# http://link.springer.com/article/
# http://onlinelibrary.wiley.com/doi/

sub_strings <- "http://dx.doi.org/|https://doi.org/|http://link.springer.com/article/|http://onlinelibrary.wiley.com/doi/"

# Remove the substrings publications$ScientificPublication_DOI
publications$ScientificPublication_DOI <- sub(sub_strings, "", publications$ScientificPublication_DOI)

In [14]:
# Split the data frame into batches of 50 rows in order to retrieve citations per publication from Crossref in batches of 50
publication_batches <- split(publications, rep(1:ceiling(nrow(publications)/50), each = 50, length.out = nrow(publications)))

In [15]:
# How many batches of 50 do we have?
end_index <- length(publication_batches)
end_index

[1] 633

In [16]:
# Prepare a bucket to append each batch's citation data
bucket <- tibble(doi = character(), is.referenced.by.count = character())

# Write an empty file with today's date in the name to keep the citations results
date_string <- format(Sys.Date(), "%Y%m%d")
file_name <- sprintf("3_PublicationCitations_%s.csv", date_string)
write.table(bucket, file_name, sep=";", fileEncoding="UTF-8", row.names = FALSE)

In [17]:
start_index <- 1

# Loop from 1 to the number of batches of 50
# To avoid timeouts at the Crossref server, there are quite some system delays implemented in this loop.
# ...also, the Crossref API is not too fast. For 600 batches of 50 DOIs, this loop runs probabyl 10 to 11 hours!!!

for (i in start_index:end_index) {
    
    # tryCatch to deal with Crossref server timeouts
    tryCatch({
        
        # The bucket for the current batch of DOIs
        batch_bucket <- tibble(doi = character(), is.referenced.by.count = character())
        
        # The current batch index as a string
        idx <- sprintf("%s", i)
      
        # The DOIs of the current batch of 50
        dois <- publication_batches[i][[idx]]$ScientificPublication_DOI
      
        # Print some progress to the console
        cat(sprintf("-%s", i))
        flush.console()
      
        # Search for records in Crossref
        results <- cr_works(dois)
        citation_counts <- select(results$data, c(doi, is.referenced.by.count))
        
        # Append to current bucket
        batch_bucket <- rbind(batch_bucket, citation_counts)
        
        # Write results so far to a file, in case of a desaster
        write.table(batch_bucket, file_name, sep = ";", append = TRUE, row.names = FALSE, col.names = FALSE)

        # 30 seconds delay before the next request  
        Sys.sleep(30)
      
  }, error = function(e) {
        
        # Print error message
        cat(sprintf("-%s", e$message))
        flush.console()
      
        next_index <- i + 1
      
        if (next_index <= end_index) {
            # If the loop isn't done yet, restart at the last index
            # ...after a 60 seconds delay to avoid running into the next timeout by the Crossref server
            Sys.sleep(60)
            i <- next_index - 1
        } else {
            stop(e)
        }
  })
}

print("Loop done")

-1

Warning message:
"404 (client error): /works/10.1074/jbc.M114.570879. - Resource not found."


-2-3-4-5-6

Warning message:
"404 (client error): /works/10.1371/journal.pone.019785 - Resource not found."


-7-8

...

-621-622-623-624-625-626-627-628-629

Warning message:
"404 (client error): /works/10.1038 - Resource not found."


-630-631-632

Warning message:
"404 (client error): /works/10.ann/rscb-2015-0044:rscb - Resource not found."


-633[1] "Loop done"


In [18]:
# Work with the final bucket
file_name.2 <- "3_PublicationCitations_20230516.csv"
publication_citations <- read_delim(file_name.2, delim = ";", col_names=TRUE)

Rows: 123591 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr (1): doi
dbl (1): is.referenced.by.count

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [19]:
# Make unique
publication_citations <- unique(publication_citations)

In [20]:
head(publication_citations)

doi,is.referenced.by.count
<chr>,<dbl>
10.1016/j.sbi.2018.03.017,30
10.3791/58109,5
10.1371/journal.pcbi.1004848,48
10.1007/s00204-017-1994-x,22
10.1016/j.neuron.2014.11.024,56
10.1016/j.celrep.2017.10.004,30


In [21]:
head(publications)

GrantNumber,ScientificPublication_DOI,ScientificPublication_Year
<dbl>,<chr>,<dbl>
173211,10.1007/s40262-019-00821-w,2020
179247,10.1038/s41598-020-60709-9,2020
150758,10.1038/mi.2014.57,2015
150758,10.1016/j.immuni.2018.12.020,2019
150758,10.1038/mi.2017.2,2017
150758,10.4414/smw.2016.14350,2016


In [22]:
# Merge publications dataframe with the citation data, order by GrantNumber, and reorder columns
citation_data <- merge(publications, publication_citations, by.x="ScientificPublication_DOI", by.y="doi", all=TRUE)
citation_data <- arrange(citation_data, GrantNumber)
citation_data <- select(citation_data, GrantNumber, ScientificPublication_DOI, ScientificPublication_Year, is.referenced.by.count)

In [23]:
head(citation_data)

,GrantNumber,ScientificPublication_DOI,ScientificPublication_Year,is.referenced.by.count
,<dbl>,<chr>,<dbl>,<dbl>
1,138726,10.1016/j.jsv.2014.05.024,2015,8
2,139011,10.1016/j.theriogenology.2014.11.030,2015,11
3,139011,10.1242/jeb.133843,2016,8
4,139011,10.3389/fevo.2016.00067,2016,6
5,139013,10.1016/j.cub.2014.01.056,2014,271
6,139013,10.1016/j.cub.2015.05.058,2015,62


In [24]:
# Group the citation_data by GrantNumber and calculate the total number of citations per grant as well as the unique number of cited publications
summarized_citations <- summarise(group_by(citation_data, GrantNumber), 
                                  NrCitablePublications=n_distinct(ScientificPublication_DOI),
                                  NrCitationsTotal=sum(is.referenced.by.count),
                                  YearFirstCitablePublication=min(ScientificPublication_Year)
                                 )

In [25]:
head(summarized_citations)

GrantNumber,NrCitablePublications,NrCitationsTotal,YearFirstCitablePublication
<dbl>,<int>,<dbl>,<dbl>
138726,1,8,2015
139011,3,25,2015
139013,14,1025,2013
139021,94,NA,2013
139093,23,1488,2015
139094,9,904,NA


In [26]:
# Load the prepared subset
biomed_grants <- read_delim("2_BiomedGrantsSubset.csv", delim = ";", col_names=TRUE)

Rows: 7609 Columns: 44
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr   (7): State, FundingInstrumentLevel1, ResearchInstitution, InstituteCou...
dbl  (32): GrantNumber, CallDecisionYear, GrantDurationDays, GrantDurationYe...
date  (5): EffectiveGrantStartDate, EffectiveGrantEndDate, DateFirstAcademic...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [27]:
# Merge summarized citation data with the grant data of interest
biomed_grant_data <- left_join(biomed_grants, summarized_citations, by="GrantNumber")

In [28]:
biomed_grant_data <- arrange(biomed_grant_data, CallDecisionYear)

In [29]:
head(biomed_grant_data)

GrantNumber,CallDecisionYear,EffectiveGrantStartDate,GrantDurationDays,GrantDurationYears,EffectiveGrantEndDate,State,FundingInstrumentLevel1,AmountGrantedAllSets,AmountPerYear,AmountPerEmployee,AmountPerEmployeeAndYear,ResearchInstitution,InstituteCountry,MainDiscipline_Level1,MainDiscipline_Level2,MainDiscipline,NrEmployees,NrEmployeesPerYear,NrCollaborations,NrCollaborationsPerYear,NrAcademicEvents,NrAcademicEventsPerYear,DateFirstAcademicEvent,NrAwards,NrAwardsPerYear,YearFirstAward,NrDatasets,NrDatasetsPerYear,DateFirstDataset,NrKnowledgeTransfer,NrKnowledgeTransferPerYear,DateFirstKnowledgeTransfer,NrCommunication,NrCommunicationPerYear,YearFirstCommunication,NrTotalPublications,NrTotalPublicationsPerYear,YearFirstPublication,NrUses,NrUsesPerYear,YearFirstUse,NrTotalOutputs,NrTotalOutputsPerYear,NrCitablePublications,NrCitationsTotal,YearFirstCitablePublication
<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
142628,2012,2013-09-01,1490,4.0821918,2017-09-30,Completed,Careers,578812,141789.52,192937.33,47263.17,University of Lausanne - LA,Switzerland,Biology and Medicine,General Biology,Ecology,3,0.7348993,2,0.4899329,4,0.9798658,2015-03-10,1,0.2449664,2013,0,0,NA,0,0,NA,0,0.0000000,NA,8,1.9597315,2014,1,0.2449664,2015,14,3.429530,7,293,2014
142615,2012,2013-04-01,1095,3.0000000,2016-03-31,Completed,Careers,534863,178287.67,267431.50,89143.83,ETH Zurich - ETHZ,Switzerland,Biology and Medicine,Basic Biological Research,"Cellular Biology, Cytology",2,0.6666667,9,3.0000000,16,5.3333333,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,1,0.3333333,2013,8,2.6666667,NA,0,0.0000000,NA,25,8.333333,6,NA,NA
144100,2012,2013-09-01,1641,4.4958904,2018-02-28,Completed,Projects,291191,64768.26,72797.75,16192.07,University of Berne - BE,Switzerland,Biology and Medicine,Preventive Medicine (Epidemiology/Early Diagnosis/Prevention),"Mental Disorders, Psychosomatic Diseases",4,0.8897014,6,1.3345521,43,9.5642901,2014-03-01,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,4,0.8897014,NA,0,0.0000000,NA,47,10.453991,3,145,2015
147291,2012,2013-02-01,364,0.9972603,2014-01-31,Completed,Careers,60400,60565.93,0.00,0.00,Institution abroad - United States of America,United States of America,Biology and Medicine,Basic Biological Research,Biochemistry,0,0.0000000,1,1.0027473,1,1.0027473,2013-05-06,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,0,0.0000000,NA,0,0.0000000,NA,1,1.002747,NA,NA,NA
142408,2012,2013-01-01,545,1.4931507,2014-06-30,Completed,Careers,584267,391298.08,584267.00,391298.08,University of Zurich - ZH,Switzerland,Biology and Medicine,Basic Biological Research,Experimental Microbiology,1,0.6697248,4,2.6788991,4,2.6788991,2013-05-07,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,2,1.3394495,2013,0,0.0000000,NA,6,4.018349,1,NA,2013
143733,2012,2013-01-01,1276,3.4958904,2016-06-30,Completed,Projects,500000,143025.08,125000.00,35756.27,University of Geneva - GE,Switzerland,Biology and Medicine,Basic Biological Research,Biochemistry,4,1.1442006,6,1.7163009,18,5.1489028,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,6,1.7163009,2013,9,2.5744514,NA,0,0.0000000,NA,33,9.439655,8,216,NA


In [30]:
# Add number of uncitable publications (no DOI)
biomed_grant_data <- mutate(biomed_grant_data, NrUncitablePublications=NrTotalPublications-NrCitablePublications, .after=YearFirstPublication)

# Add number of uncitable publications (no DOI) per year
biomed_grant_data <- mutate(biomed_grant_data, NrUncitablePublicationsPerYear=NrUncitablePublications/GrantDurationYears, .after=NrUncitablePublications)

# Add number of citable publications per year
biomed_grant_data <- mutate(biomed_grant_data, NrCitablePublicationsPerYear=NrCitablePublications/GrantDurationYears, .after=NrCitablePublications)

# Add number of citations per citable publication
biomed_grant_data <- mutate(biomed_grant_data, NrCitationsPerPublication=NrCitationsTotal/NrCitablePublications, .after=NrCitationsTotal)

In [31]:
head(biomed_grant_data)

GrantNumber,CallDecisionYear,EffectiveGrantStartDate,GrantDurationDays,GrantDurationYears,EffectiveGrantEndDate,State,FundingInstrumentLevel1,AmountGrantedAllSets,AmountPerYear,AmountPerEmployee,AmountPerEmployeeAndYear,ResearchInstitution,InstituteCountry,MainDiscipline_Level1,MainDiscipline_Level2,MainDiscipline,NrEmployees,NrEmployeesPerYear,NrCollaborations,NrCollaborationsPerYear,NrAcademicEvents,NrAcademicEventsPerYear,DateFirstAcademicEvent,NrAwards,NrAwardsPerYear,YearFirstAward,NrDatasets,NrDatasetsPerYear,DateFirstDataset,NrKnowledgeTransfer,NrKnowledgeTransferPerYear,DateFirstKnowledgeTransfer,NrCommunication,NrCommunicationPerYear,YearFirstCommunication,NrTotalPublications,NrTotalPublicationsPerYear,YearFirstPublication,NrUncitablePublications,NrUncitablePublicationsPerYear,NrUses,NrUsesPerYear,YearFirstUse,NrTotalOutputs,NrTotalOutputsPerYear,NrCitablePublications,NrCitablePublicationsPerYear,NrCitationsTotal,NrCitationsPerPublication,YearFirstCitablePublication
<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
142628,2012,2013-09-01,1490,4.0821918,2017-09-30,Completed,Careers,578812,141789.52,192937.33,47263.17,University of Lausanne - LA,Switzerland,Biology and Medicine,General Biology,Ecology,3,0.7348993,2,0.4899329,4,0.9798658,2015-03-10,1,0.2449664,2013,0,0,NA,0,0,NA,0,0.0000000,NA,8,1.9597315,2014,1,0.2449664,1,0.2449664,2015,14,3.429530,7,1.7147651,293,41.85714,2014
142615,2012,2013-04-01,1095,3.0000000,2016-03-31,Completed,Careers,534863,178287.67,267431.50,89143.83,ETH Zurich - ETHZ,Switzerland,Biology and Medicine,Basic Biological Research,"Cellular Biology, Cytology",2,0.6666667,9,3.0000000,16,5.3333333,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,1,0.3333333,2013,8,2.6666667,NA,2,0.6666667,0,0.0000000,NA,25,8.333333,6,2.0000000,NA,NA,NA
144100,2012,2013-09-01,1641,4.4958904,2018-02-28,Completed,Projects,291191,64768.26,72797.75,16192.07,University of Berne - BE,Switzerland,Biology and Medicine,Preventive Medicine (Epidemiology/Early Diagnosis/Prevention),"Mental Disorders, Psychosomatic Diseases",4,0.8897014,6,1.3345521,43,9.5642901,2014-03-01,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,4,0.8897014,NA,1,0.2224254,0,0.0000000,NA,47,10.453991,3,0.6672761,145,48.33333,2015
147291,2012,2013-02-01,364,0.9972603,2014-01-31,Completed,Careers,60400,60565.93,0.00,0.00,Institution abroad - United States of America,United States of America,Biology and Medicine,Basic Biological Research,Biochemistry,0,0.0000000,1,1.0027473,1,1.0027473,2013-05-06,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,0,0.0000000,NA,NA,NA,0,0.0000000,NA,1,1.002747,NA,NA,NA,NA,NA
142408,2012,2013-01-01,545,1.4931507,2014-06-30,Completed,Careers,584267,391298.08,584267.00,391298.08,University of Zurich - ZH,Switzerland,Biology and Medicine,Basic Biological Research,Experimental Microbiology,1,0.6697248,4,2.6788991,4,2.6788991,2013-05-07,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,2,1.3394495,2013,1,0.6697248,0,0.0000000,NA,6,4.018349,1,0.6697248,NA,NA,2013
143733,2012,2013-01-01,1276,3.4958904,2016-06-30,Completed,Projects,500000,143025.08,125000.00,35756.27,University of Geneva - GE,Switzerland,Biology and Medicine,Basic Biological Research,Biochemistry,4,1.1442006,6,1.7163009,18,5.1489028,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,6,1.7163009,2013,9,2.5744514,NA,1,0.2860502,0,0.0000000,NA,33,9.439655,8,2.2884013,216,27.00000,NA


In [32]:
# NA --> 0 in certain columns
columns_to_mutate <- c(
  "NrCitablePublications",
  "NrCitablePublicationsPerYear",
  "NrUncitablePublications",
  "NrUncitablePublicationsPerYear"
)

biomed_grant_data <- mutate(biomed_grant_data, across(all_of(columns_to_mutate), ~ ifelse(is.na(.), 0, .)))

In [33]:
# Add the total number of uncitable outputs (outputs other than peer-reviewed articles with a DOI)
biomed_grant_data <- mutate(biomed_grant_data, NrTotalUncitableOutputs=NrTotalOutputs - NrCitablePublications, .after=NrTotalOutputsPerYear)

# Total number of uncitable outputs per year
biomed_grant_data <- mutate(biomed_grant_data, NrTotalUncitableOutputsPerYear=NrTotalUncitableOutputs/GrantDurationYears, .after=NrTotalUncitableOutputs)

In [34]:
dim(biomed_grant_data)
head(biomed_grant_data)

[1] 7609   53

GrantNumber,CallDecisionYear,EffectiveGrantStartDate,GrantDurationDays,GrantDurationYears,EffectiveGrantEndDate,State,FundingInstrumentLevel1,AmountGrantedAllSets,AmountPerYear,AmountPerEmployee,AmountPerEmployeeAndYear,ResearchInstitution,InstituteCountry,MainDiscipline_Level1,MainDiscipline_Level2,MainDiscipline,NrEmployees,NrEmployeesPerYear,NrCollaborations,NrCollaborationsPerYear,NrAcademicEvents,NrAcademicEventsPerYear,DateFirstAcademicEvent,NrAwards,NrAwardsPerYear,YearFirstAward,NrDatasets,NrDatasetsPerYear,DateFirstDataset,NrKnowledgeTransfer,NrKnowledgeTransferPerYear,DateFirstKnowledgeTransfer,NrCommunication,NrCommunicationPerYear,YearFirstCommunication,NrTotalPublications,NrTotalPublicationsPerYear,YearFirstPublication,NrUncitablePublications,NrUncitablePublicationsPerYear,NrUses,NrUsesPerYear,YearFirstUse,NrTotalOutputs,NrTotalOutputsPerYear,NrTotalUncitableOutputs,NrTotalUncitableOutputsPerYear,NrCitablePublications,NrCitablePublicationsPerYear,NrCitationsTotal,NrCitationsPerPublication,YearFirstCitablePublication
<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
142628,2012,2013-09-01,1490,4.0821918,2017-09-30,Completed,Careers,578812,141789.52,192937.33,47263.17,University of Lausanne - LA,Switzerland,Biology and Medicine,General Biology,Ecology,3,0.7348993,2,0.4899329,4,0.9798658,2015-03-10,1,0.2449664,2013,0,0,NA,0,0,NA,0,0.0000000,NA,8,1.9597315,2014,1,0.2449664,1,0.2449664,2015,14,3.429530,7,1.714765,7,1.7147651,293,41.85714,2014
142615,2012,2013-04-01,1095,3.0000000,2016-03-31,Completed,Careers,534863,178287.67,267431.50,89143.83,ETH Zurich - ETHZ,Switzerland,Biology and Medicine,Basic Biological Research,"Cellular Biology, Cytology",2,0.6666667,9,3.0000000,16,5.3333333,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,1,0.3333333,2013,8,2.6666667,NA,2,0.6666667,0,0.0000000,NA,25,8.333333,19,6.333333,6,2.0000000,NA,NA,NA
144100,2012,2013-09-01,1641,4.4958904,2018-02-28,Completed,Projects,291191,64768.26,72797.75,16192.07,University of Berne - BE,Switzerland,Biology and Medicine,Preventive Medicine (Epidemiology/Early Diagnosis/Prevention),"Mental Disorders, Psychosomatic Diseases",4,0.8897014,6,1.3345521,43,9.5642901,2014-03-01,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,4,0.8897014,NA,1,0.2224254,0,0.0000000,NA,47,10.453991,44,9.786715,3,0.6672761,145,48.33333,2015
147291,2012,2013-02-01,364,0.9972603,2014-01-31,Completed,Careers,60400,60565.93,0.00,0.00,Institution abroad - United States of America,United States of America,Biology and Medicine,Basic Biological Research,Biochemistry,0,0.0000000,1,1.0027473,1,1.0027473,2013-05-06,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,0,0.0000000,NA,0,0.0000000,0,0.0000000,NA,1,1.002747,1,1.002747,0,0.0000000,NA,NA,NA
142408,2012,2013-01-01,545,1.4931507,2014-06-30,Completed,Careers,584267,391298.08,584267.00,391298.08,University of Zurich - ZH,Switzerland,Biology and Medicine,Basic Biological Research,Experimental Microbiology,1,0.6697248,4,2.6788991,4,2.6788991,2013-05-07,0,0.0000000,NA,0,0,NA,0,0,NA,0,0.0000000,NA,2,1.3394495,2013,1,0.6697248,0,0.0000000,NA,6,4.018349,5,3.348624,1,0.6697248,NA,NA,2013
143733,2012,2013-01-01,1276,3.4958904,2016-06-30,Completed,Projects,500000,143025.08,125000.00,35756.27,University of Geneva - GE,Switzerland,Biology and Medicine,Basic Biological Research,Biochemistry,4,1.1442006,6,1.7163009,18,5.1489028,2013-05-16,0,0.0000000,NA,0,0,NA,0,0,NA,6,1.7163009,2013,9,2.5744514,NA,1,0.2860502,0,0.0000000,NA,33,9.439655,25,7.151254,8,2.2884013,216,27.00000,NA


In [35]:
# Write  data to file
date_string <- format(Sys.Date(), "%Y%m%d")
file_name.3 <- sprintf("3_BiomedGrantData_%s.csv", date_string)

# Into tidy
write.table(biomed_grant_data, file_name.3, sep=";", fileEncoding="UTF-8", row.names = FALSE)